### Sentiment analysis of movie (IMDB) reviews using dataset provided by the ACL 2011 paper, see http://ai.stanford.edu/~amaas/data/sentiment/.

#### Dataset can be downloaded separately from http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz, but wont be necessary as the download process has been embedded in the notebook and source file.

In [ ]:
!pip install nltk
!pip install --upgrade gensim

import numpy as np
import os
import os.path

from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
import nltk


import glob
from gensim.models import Word2Vec

import time
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# MacOSX: See https://www.mkyong.com/mac/wget-on-mac-os-x/ for wget
if not os.path.isdir('aclImdb'):
    if not os.path.isfile('aclImdb_v1.tar.gz'):
      !wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz 

    if not os.path.isdir('aclImdb'):  
      !tar -xf aclImdb_v1.tar.gz 

In [ ]:
time_beginning_of_notebook = time.time()
SAMPLE_SIZE=1000
positive_sample_file_list = glob.glob(os.path.join('aclImdb/train/pos', "*.txt"))
positive_sample_file_list = positive_sample_file_list[:SAMPLE_SIZE]

negative_sample_file_list = glob.glob(os.path.join('aclImdb/train/neg', "*.txt"))
negative_sample_file_list = negative_sample_file_list[:SAMPLE_SIZE]

import re

# load doc into memory
# regex to clean markup elements 
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r', encoding='utf8')
    # read all text
    text = re.sub('<[^>]*>', ' ', file.read())
    #text = file.read()
    # close the file
    file.close()
    return text


# Data exploration

In [ ]:
positive_strings = [load_doc(x) for x in positive_sample_file_list]
negative_strings = [load_doc(x) for x in negative_sample_file_list]

In [ ]:
positive_tokenized = [word_tokenize(s) for s in positive_strings]
negative_tokenized = [word_tokenize(s) for s in negative_strings]

In [ ]:
# load doc into memory
with open('aclImdb/imdb.vocab', encoding='utf8') as f:
    #content = f.readlines()
    universe_vocabulary = [x.strip() for x in f.readlines()]

print("Word count across all reviews (before stripping tokens):", sum([len(token) for token in positive_tokenized]))

#Checking the not alphanumeric characters in vocabulary
non_alphanumeric_set = set()
for word in universe_vocabulary:
    non_alphanumeric_set |= set(re.findall('\W', word))
print('Non alphanumeric characters found in universe vocabulary', non_alphanumeric_set)


stripped_positive_tokenized = []
for tokens in positive_tokenized:
  stripped_positive_tokenized.append([token.lower() for token in tokens if token.lower() in universe_vocabulary])

print("Word count across all reviews (after stripping tokens):", sum([len(token) for token in stripped_positive_tokenized]))

In [ ]:
print("Word count across all reviews (before stripping tokens):", sum([len(token) for token in positive_tokenized]))
stripped_negative_tokenized = []
for tokens in negative_tokenized:
  stripped_negative_tokenized.append([token.lower() for token in tokens if token.lower() in universe_vocabulary])

print("Word count across all reviews (after stripping tokens):", sum([len(token) for token in stripped_negative_tokenized]))

## Modelling 

We have decided to do the use the below models and vectorisation techniques to test our their accuracy / score, the idea is to use a one model and one vectorization technique and plot a score.

**Simple models**

- Logistic Regression
- Random Forst
- LSTM
- GRU
- CNN

**Vectorisation techniques**
- Bag of Words
- Word2Vec
- TFIDF (probability scores)
- FastText
- Glove

## Logistic Regression
### Introducing Pipeline: 
http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
### Introducing TfdfVectorizer: 
http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
### Introducing cross_val_score 
http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

In [ ]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

df_positives = pd.DataFrame({'reviews':[load_doc(x) for x in positive_sample_file_list], 'sentiment': np.ones(SAMPLE_SIZE)})
df_negatives = pd.DataFrame({'reviews':[load_doc(x) for x in negative_sample_file_list], 'sentiment': np.zeros(SAMPLE_SIZE)})

print("Positive review(s):", df_positives['reviews'][1][:100])
print("Negative review(s):", df_negatives['reviews'][1][:100])

df = pd.concat([df_positives, df_negatives], ignore_index=True)

df = shuffle(df)

X_train, X_test, y_train, y_test = train_test_split(df['reviews'], df['sentiment'], test_size=0.25)

### Creating the BoW with the set of all the documents and transforming the documents in feature vectors

In [ ]:
CountVec = CountVectorizer()
bag_of_words = CountVec.fit_transform(df['reviews'])
print(type(bag_of_words))
print('\n Number of rows {} (documents) -- Number of columns {} (vocabulary) \n'.format(bag_of_words.shape[0], bag_of_words.shape[1]))
print('\n Type of bag_of_words {} \n'.format(type(bag_of_words)))
sparsity = 1.0 - bag_of_words.nnz / (bag_of_words.shape[0] * bag_of_words.shape[1])
print('\n Sparsity {} \n'.format(sparsity))

print('\n Type of bag_of_words.toarray {} \n'.format(type(bag_of_words.toarray())))

print('\n Type of CountVec.vocabulary {} \n'.format(type(CountVec.vocabulary_)))
print('A sample of CountVec.vocabulary_ {}'.format([(k, v) for k, v in CountVec.vocabulary_.items() if v < 1000]))
print('\n Type of bag_of_words.toarray() {} \n')
print('\n First feature vector, representing the first document \n', bag_of_words[0, :])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(bag_of_words.toarray(), df['sentiment'].values, test_size=0.25)

clf = LogisticRegression(random_state=0)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
results = [(predicted, actual) for predicted, actual in zip(clf.predict(X_test),  y_test) 
           if  predicted == actual]

print('Percentage of correct predicted values{}'.format(len(results)/len(y_test)))

## Logistic Regress model using Bag of Words vectorisation technique

In [ ]:
X_train

In [ ]:
countvec = CountVectorizer()
lr_CV = Pipeline([('vect', countvec), ('tfidf', TfidfTransformer()), ('clf', LogisticRegression(random_state=0))])
lr_CV.fit(X_train, y_train)
print('Train accuracy {}'.format(lr_CV.score(X_train, y_train)))
print('Test accuracy {}'.format(lr_CV.score(X_test, y_test)))

# Trying with cross_val_score
lr = LogisticRegression()
k_folds = 10
X_train_CV = countvec.fit_transform(X_train)
type(X_train_CV)
print('Train accuracy list {} '.format(cross_val_score(lr, X_train_CV, y_train, cv= k_folds))) 
print('Train accuracy mean {} '.format(cross_val_score(lr, X_train_CV, y_train, cv= k_folds).mean()))

## Logistic Regress model using TfidfVectorizer vectorisation technique

In [48]:
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)

lr_tfidf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(random_state=0))])
lr_tfidf.fit(X_train, y_train)
print('Train accuracy {}'.format(lr_tfidf.score(X_train, y_train)))
print('Test accuracy {}'.format(lr_tfidf.score(X_test, y_test)))

# Trying with cross_val_score
lr = LogisticRegression()
k_folds = 10
X_train_tfidf = tfidf.fit_transform(X_train)
print('Train accuracy list {} '.format(cross_val_score(lr, X_train_tfidf, y_train, cv= k_folds))) 
print('Train accuracy mean {} '.format(cross_val_score(lr, X_train_tfidf, y_train, cv= k_folds).mean()))

Train accuracy 0.97
Test accuracy 0.826
Train accuracy list [0.8410596  0.81456954 0.80794702 0.84768212 0.75333333 0.84
 0.7852349  0.83892617 0.80536913 0.81879195] 
Train accuracy mean 0.8152913759130035 


## Logistic Regress model using TfidfVectorizer and different values for C hyperparameter

In [28]:
C_values = np.arange(1,2,0.1)
results = []

for value in C_values:   
    lr_tfidf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(random_state=0, C=value))])
    lr_tfidf.fit(X_train, y_train)
    train_score = lr_tfidf.score(X_train, y_train)
    score = lr_tfidf.score(X_test, y_test)
    print('C_value {} Test Score {} Train_score {}'.format(value, score, train_score))
    results.append(score)

time_end_of_notebook = time.time()

C_value 1.0 Test Score 0.832 Train_score 0.9686666666666667
C_value 1.1 Test Score 0.834 Train_score 0.9733333333333334
C_value 1.2000000000000002 Test Score 0.834 Train_score 0.9786666666666667
C_value 1.3000000000000003 Test Score 0.834 Train_score 0.98
C_value 1.4000000000000004 Test Score 0.834 Train_score 0.9826666666666667
C_value 1.5000000000000004 Test Score 0.834 Train_score 0.9846666666666667
C_value 1.6000000000000005 Test Score 0.832 Train_score 0.9853333333333333
C_value 1.7000000000000006 Test Score 0.834 Train_score 0.9866666666666667
C_value 1.8000000000000007 Test Score 0.836 Train_score 0.988
C_value 1.9000000000000008 Test Score 0.836 Train_score 0.988


In [29]:
table_models_vectorization = pd.DataFrame(
     {'Models':                   ["Logistic Regression", "Logistic Regression", "Logistic Regression"], 
      'Vectorisation techniques': ["Bag of Words",        "Word2Vec", "TFIDF"], 
      'Score':                    [score,                 "Pending", lr_tfidf.score(X_train, y_train) ]},
    columns=['Models','Vectorisation techniques','Score']
)
print("Sample size:", SAMPLE_SIZE)

duration = time_end_of_notebook - time_beginning_of_notebook

print("Full notebook execution duration:", duration, "seconds")
print("Full notebook execution duration:", duration / 60, "minutes")

table_models_vectorization

Sample size: 1000
Full notebook execution duration: 6434.627096891403 seconds
Full notebook execution duration: 107.24378494819005 minutes


,Models,Vectorisation techniques,Score
0,Logistic Regression,Bag of Words,0.836
1,Logistic Regression,Word2Vec,Pending
2,Logistic Regression,TFIDF,0.988


**The below two code blocks replaces the original/inital BoW implementation using Scikit-learn**

In [30]:
CountVec = CountVectorizer()
# Creating the BoW with the set of all the documents and transforming the documents in feature vectors
bag_of_words = CountVec.fit_transform(df['reviews'])

print(type(bag_of_words))
print('\n Number of raws {} (documents) -- Number of columns {} (vocabulary) \n'.format(bag_of_words.shape[0], bag_of_words.shape[1]))

# https://machinelearningmastery.com/sparse-matrices-for-machine-learning/
# This is a sparse matrix 
print('\n Type of bag_of_words {} \n'.format(type(bag_of_words)))
sparsity = 1.0 - bag_of_words.nnz / (bag_of_words.shape[0] * bag_of_words.shape[1])
print('\n Sparsity {} \n'.format(sparsity))

# This is a  
print('\n Type of bag_of_words.toarray {} \n'.format(type(bag_of_words.toarray())))

# In CountVec we have the vocabulary as an attribute
print('\n Type of CountVec.vocabulary {} \n'.format(type(CountVec.vocabulary_)))
print('A sample of CountVec.vocabulary_ {}'.format([(k, v) for k, v in CountVec.vocabulary_.items() if v < 1000]))

# In bag_of_words we have the vector features representing each single document 
print('\n Type of bag_of_words.toarray() {} \n')
print('\n First feature vector, representing the first document \n', bag_of_words[0, :2])

<class 'scipy.sparse.csr.csr_matrix'>

 Number of raws 2000 (documents) -- Number of columns 25644 (vocabulary) 


 Type of bag_of_words <class 'scipy.sparse.csr.csr_matrix'> 


 Sparsity 0.9946127554203712 


 Type of bag_of_words.toarray <class 'numpy.ndarray'> 


 Type of CountVec.vocabulary <class 'dict'> 

A sample of CountVec.vocabulary_ [('after', 710), ('45', 244), ('50', 253), ('acting', 512), ('all', 885), ('absurd', 400), ('1922', 87), ('40', 237), ('also', 949), ('1939', 102), ('already', 946), ('1935', 98), ('africa', 706), ('1930', 92), ('allah', 886), ('alan', 823), ('135', 33), ('1970s', 136), ('aliens', 877), ('agents', 733), ('about', 380), ('10', 12), ('absolutely', 392), ('ability', 361), ('act', 510), ('acts', 527), ('1983', 149), ('age', 727), ('although', 967), ('advise', 660), ('able', 365), ('40s', 239), ('adrenalin', 620), ('actors', 524), ('666', 273), ('60', 266), ('always', 976), ('add', 547), ('ah', 765), ('alcoholic', 842), ('alcohol', 841), ('48', 247), 

In [50]:
# Lets get our training and test data
X_train, X_test, y_train, y_test = train_test_split(bag_of_words.toarray(), df['sentiment'].values, test_size=0.25)


In [51]:
clf = LogisticRegression(random_state=0)
clf.fit(X_train, y_train)
print('Using score function: {}'.format(clf.score(X_test, y_test)))
results = [(predicted, actual) for predicted, actual in zip(clf.predict(X_test),  y_test) 
           if  predicted == actual]
print('Percentage of correct predicted values: {}'.format(len(results)/len(y_test)))

Using score function: 0.824
Percentage of correct predicted values: 0.824


### RandomForestClassifer

In [52]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=1000)
clf.fit(X_train, y_train)
print('Using score function: {}'.format(clf.score(X_test, y_test)))


results = [(predicted, actual) for predicted, actual in zip(clf.predict(X_test),  y_test) 
           if  predicted == actual]

print('Percentage of correct predicted values: {}'.format(len(results)/len(y_test)))

Using score function: 0.828
Percentage of correct predicted values: 0.828


### RandomForestClassifer with TFIDF (Pipeline)

In [53]:
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)


X_train, X_test, y_train, y_test = train_test_split(df['reviews'], df['sentiment'], test_size=0.25)

lr_tfidf = Pipeline([('vect', tfidf), ('clf', RandomForestClassifier(n_estimators=1000))])
lr_tfidf.fit(X_train, y_train)
print('Train accuracy {}'.format(lr_tfidf.score(X_train, y_train)))
print('Test accuracy {}'.format(lr_tfidf.score(X_test, y_test)))

# Trying with cross_val_score
lr = LogisticRegression()
k_folds = 10
X_train_tfidf = tfidf.fit_transform(X_train)
print('Train accuracy list {} '.format(cross_val_score(lr, X_train_tfidf, y_train, cv= k_folds))) 
print('Train accuracy mean {} '.format(cross_val_score(lr, X_train_tfidf, y_train, cv= k_folds).mean()))


Train accuracy 1.0
Test accuracy 0.816
Train accuracy list [0.84768212 0.86092715 0.82119205 0.74834437 0.83333333 0.83333333
 0.85234899 0.8590604  0.79194631 0.80536913] 
Train accuracy mean 0.825353719424567 


### LSTM with Keras (Sequential model)


Please note that the below code is executed on GPU instances on Colab, this wont work on your local machine, use the flag to enable/disable running in CPU or GPU mode, set `run_in_GPU_mode_on_colab=false` in order to be able to run in CPU mode.

In [56]:
import tensorflow as tf

def lstm_keras():
  from keras.models import Sequential
  from keras.layers import Dense, Activation, Embedding, LSTM
  from keras.preprocessing.text import Tokenizer
  from sklearn.preprocessing import LabelBinarizer


  X_train, X_test, y_train, y_test = train_test_split(df['reviews'], df['sentiment'], test_size=0.25)

  vocab_size = 1000
  tokenize = Tokenizer(num_words=vocab_size)
  tokenize.fit_on_texts(X_train)

  encoded_X_train = tokenize.texts_to_matrix(X_train)
  encoded_X_test = tokenize.texts_to_matrix(X_test)

  encoder = LabelBinarizer()
  encoder.fit(y_train)
  encoded_y_train = encoder.transform(y_train)
  encoded_y_test = encoder.transform(y_test)

  max_features = 1000
  model = Sequential()
  model.add(Embedding(max_features, 128, dropout=0.2))
  model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))  # try using a GRU instead, for fun
  model.add(Dense(1))
  model.add(Activation('sigmoid'))

  model.compile(loss='binary_crossentropy',
               optimizer='adam',
               metrics=['accuracy'])

  batch_size=64
  epochs=10
  history = model.fit(encoded_X_train, encoded_y_train, 
                  batch_size=batch_size, 
                  epochs=epochs, 
                  verbose=1, 
                  validation_split=0.1)

  score = model.evaluate(encoded_X_test, encoded_y_test, 
                         batch_size=batch_size, verbose=1)
  print('Test score:', score[0])
  print('Test accuracy:', score[1])


run_in_GPU_mode_on_colab=False

if run_in_GPU_mode_on_colab:  
  config = tf.ConfigProto()
  config.gpu_options.allow_growth = True

  with tf.device('/gpu:0'):
    session_gpu = tf.Session(config=config)
    session_gpu.run(tf.global_variables_initializer())
    session_gpu.run(tf.tables_initializer())
    start = time.time()
    session_gpu.run(lstm_keras())
    end = time.time()
    gpu_time = end - start
    print('Duration on the GPU: {} seconds'.format(gpu_time))
else:
  start = time.time()
  lstm_keras()
  end = time.time()
  cpu_time = end - start
  print('Duration on the CPU: {} seconds'.format(cpu_time))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.


Train on 1350 samples, validate on 150 samples
Epoch 1/2
1350/1350 [==============================] - 62s 46ms/step - loss: 0.6947 - acc: 0.4822 - val_loss: 0.6932 - val_acc: 0.4933
Epoch 2/2
500/500 [==============================] - 5s 10ms/step
Test score: 0.6931333312988281
Test accuracy: 0.4959999995231628
Duration on the CPU: 128.34940028190613
